# INTRODUCTION/BUSINESS PROBLEM
The  company called 'COV21'is looking to build consulting business service to restaurants providing new strategy in order to cope with COVID 19 situation. They are targeting French market, because many of restaurants have been running traditionally. With several lockdowns, some of they couldn't survive and have to close down. The 'COV21' has several ideas on takeaways, menues and combining different icons  while maintaining their authentic French classic image. One of the biggest stakeholder of 'COV21' is subsidised by French government meaning that the company has an up to date on government policy about COVID19 measurement. They are looking for a city where it has the most French restaurant. 

In addition to restaurant industry, they have primiary discussion on idea of changing business model for hotel industry, which had huge impact during the COVID19. The company has an eligibility to get financial grant depending on the success of the business of their clients(hotels). 

The problem is to find the best location in France where it has the most restaurants and hotels. Hence, they are seeking an advice from data scientist. 

# DATA&METHOD
The data scientist will use internet and Foursquare API as main source. The used datasets were either directly downloaded from open repository. For the analysis, we will be using Python as the main language. First, the we will use https://simplemaps.com/data/fr-cities to extract latitude, longtitude and population of main 471 cities in France. Then, Foursquare API will be used to retrieve venus data of those cities. We will apply K-mean method to cluster based on the most common venue. The Foursquare API url is as following: https://foursquare.com/


In [1]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6e45cef792a145a5a48d74941672fa3a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WQyLwrun1mYYpVBCkAJPKPwhR5oY4DsBnuKc64kGVTm_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_6e45cef792a145a5a48d74941672fa3a.get_object(Bucket='capstoneibmchristine-donotdelete-pr-hggep8rsmdyqwb',Key='fr.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Paris,48.8566,2.3522,France,FR,Île-de-France,primary,11020000.0,2148271.0
1,Nice,43.7034,7.2663,France,FR,Provence-Alpes-Côte d’Azur,minor,1006402.0,340017.0
2,Toulouse,43.6045,1.4440,France,FR,Occitanie,admin,968638.0,479553.0
3,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,admin,870018.0,855393.0
4,Rennes,48.1147,-1.6794,France,FR,Bretagne,admin,727357.0,216815.0


In [65]:
france_cities=df_data_1[['city', 'lat','lng','population']]
france_cities.head()

,city,lat,lng,population
0,Paris,48.8566,2.3522,11020000.0
1,Nice,43.7034,7.2663,1006402.0
2,Toulouse,43.6045,1.4440,968638.0
3,Marseille,43.2964,5.3700,870018.0
4,Rennes,48.1147,-1.6794,727357.0


In [66]:
france_cities.shape

(471, 4)

In [67]:
france_cities = france_cities.rename({"lat":"latitudes","lng":"longtitudes"}, axis='columns')
france_cities.head()

,city,latitudes,longtitudes,population
0,Paris,48.8566,2.3522,11020000.0
1,Nice,43.7034,7.2663,1006402.0
2,Toulouse,43.6045,1.4440,968638.0
3,Marseille,43.2964,5.3700,870018.0
4,Rennes,48.1147,-1.6794,727357.0


In [71]:
CLIENT_ID = 'SFP0OARWG3G1LKMFE4WD4KXIS3SL5UXQ5IMANKP2YCBOB102' # your Foursquare ID
CLIENT_SECRET = 'LJVS3Y1ORDAFOKYKV4DQZFIR3JOXL0YDAQY1WWCYU3HAB0HW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SFP0OARWG3G1LKMFE4WD4KXIS3SL5UXQ5IMANKP2YCBOB102
CLIENT_SECRET:LJVS3Y1ORDAFOKYKV4DQZFIR3JOXL0YDAQY1WWCYU3HAB0HW


In [22]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0  
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 79 kB 9.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=ad422fc5a8d5de8a21369de610c4023b58398246ee18b736def4f743214b2273
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


In [72]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
#To generate venues
france_venues = getNearbyVenues(names=france_cities['city'],
                                   latitudes=france_cities['latitudes'],
                                   longitudes=france_cities['longtitudes']
                                  )

Paris
Nice
Toulouse
Marseille
Rennes
Grenoble
Nantes
Montpellier
Lyon
Rouen
Strasbourg
Nancy
Metz
Brest
Mulhouse
Limoges
Orléans
Bordeaux
Le Havre
Lille
Reims
Saint-Étienne
Toulon
Dijon
Angers
Nîmes
Villeurbanne
Clermont-Ferrand
Le Mans
Aix-en-Provence
Amiens
Tours
Boulogne-sur-Mer
Annecy
Perpignan
Boulogne-Billancourt
Besançon
Belfort
Saint-Denis
Argenteuil
Montreuil
Caen
Roubaix
Tourcoing
Nanterre
Avignon
Vitry-sur-Seine
Épinal
Créteil
Poitiers
Dunkerque
Aubervilliers
Versailles
Aulnay-sous-Bois
Carrières-sur-Seine
Colombes
Courbevoic
Rueil-Malmaison
Champigny-sur-Marne
Béziers
Pau
La Rochelle
Saint-Maur-des-Fossés
Calais
Cannes
Antibes
Drancy
Ajaccio
Mérignac
Saint-Nazaire
Colmar
Issy-les-Moulineaux
Noisy-le-Grand
Courcouronnes
Vénissieux
Cergy
Bourges
Levallois-Perret
La Seyne-sur-Mer
Pessac
Valence
Quimper
Antony
Ivry-sur-Seine
Montélimar
Troyes
Clichy
Cherbourg
Montauban
Neuilly-sur-Seine
Chambéry
Niort
Sarcelles
Pantin
Lorient
Le Blanc-Mesnil
Beauvais
Maisons-Alfort
Hyères
Épina

In [74]:
print(france_venues.shape)
france_venues.head()

(5545, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paris,48.8566,2.3522,Place de l'Hôtel de Ville – Esplanade de la Li...,48.857010,2.351656,Plaza
1,Paris,48.8566,2.3522,Maison Aleph,48.857348,2.354873,Pastry Shop
2,Paris,48.8566,2.3522,Berges de Seine – Rive droite,48.855131,2.352289,Pedestrian Plaza
3,Paris,48.8566,2.3522,Parc Rives de Seine,48.855510,2.351419,Park
4,Paris,48.8566,2.3522,LUSH,48.857791,2.351622,Cosmetics Shop


In [75]:
france_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbeville,9,9,9,9,9,9
Agde,5,5,5,5,5,5
Agen,14,14,14,14,14,14
Aix-en-Provence,100,100,100,100,100,100
Aix-les-Bains,13,13,13,13,13,13
...,...,...,...,...,...,...
Épinal,10,10,10,10,10,10
Épinay-sur-Seine,9,9,9,9,9,9
Étampes,5,5,5,5,5,5


In [76]:
print('There are {} uniques categories.'.format(len(france_venues['Venue Category'].unique())))

There are 331 uniques categories.


In [77]:
# one hot encoding
france_onehot = pd.get_dummies(france_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
france_onehot['Neighborhood'] = france_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [france_onehot.columns[-1]] + list(france_onehot.columns[:-1])
france_onehot = france_onehot[fixed_columns]

france_onehot.head()

,Women's Store,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Watch Shop,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
france_onehot.shape

(5545, 331)

In [79]:
france_grouped = france_onehot.groupby('Neighborhood').mean().reset_index()
france_grouped

,Neighborhood,Women's Store,Accessories Store,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Watch Shop,Water Park,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery
0,Abbeville,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,Agde,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,Agen,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Aix-en-Provence,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0
4,Aix-les-Bains,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,Épinal,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
463,Épinay-sur-Seine,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
464,Étampes,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
465,Évreux,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [80]:
france_grouped.shape

(467, 331)

In [81]:
#To generate top 5 venues
num_top_venues = 5

for hood in france_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = france_grouped[france_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbeville----
            venue  freq
0           Hotel  0.22
1          Lawyer  0.11
2     Supermarket  0.11
3   Historic Site  0.11
4  Clothing Store  0.11


----Agde----
               venue  freq
0               Café   0.2
1         Smoke Shop   0.2
2          Multiplex   0.2
3  French Restaurant   0.2
4           Pharmacy   0.2


----Agen----
                 venue  freq
0            Multiplex  0.14
1  Japanese Restaurant  0.07
2                Hotel  0.07
3     Department Store  0.07
4                Plaza  0.07


----Aix-en-Provence----
               venue  freq
0  French Restaurant  0.19
1              Hotel  0.06
2              Plaza  0.06
3                Pub  0.04
4                Bar  0.04


----Aix-les-Bains----
                  venue  freq
0    Italian Restaurant  0.23
1                 Hotel  0.15
2  Medical Supply Store  0.08
3             Gastropub  0.08
4                Casino  0.08


----Ajaccio----
               venue  freq
0  French Restaurant  0.21
1       



----Blaye----
               venue  freq
0              Hotel  0.25
1          Bookstore  0.12
2          Wine Shop  0.12
3  French Restaurant  0.12
4      Garden Center  0.12


----Blois----
                   venue  freq
0                    Bar   1.0
1          Women's Store   0.0
2            Opera House   0.0
3           Outlet Store   0.0
4  Outdoors & Recreation   0.0


----Bobigny----
                   venue  freq
0   Fast Food Restaurant  0.25
1           Tram Station  0.25
2                  Hotel  0.12
3               Creperie  0.12
4  Performing Arts Venue  0.12


----Bois-Colombes----
                 venue  freq
0               Garden   0.1
1                Plaza   0.1
2  Martial Arts School   0.1
3                Beach   0.1
4                 Café   0.1


----Bondy----
                        venue  freq
0                  Shoe Store  0.12
1                   Pet Store  0.12
2           French Restaurant  0.12
3  Financial or Legal Service  0.12
4                 Supe

                 venue  freq
0        Movie Theater   0.2
1  Indie Movie Theater   0.2
2            Gastropub   0.2
3                Hotel   0.2
4                 Park   0.2


----Château-Thierry----
                venue  freq
0  Italian Restaurant   0.2
1                Café   0.2
2      History Museum   0.2
3   Mobile Phone Shop   0.2
4               Hotel   0.2


----Châteaubriant----
                        venue  freq
0  Construction & Landscaping  0.14
1                 Supermarket  0.14
2               Historic Site  0.14
3           Electronics Store  0.14
4               Train Station  0.14


----Châteaudun----
               venue  freq
0      Train Station  0.25
1  Food & Drink Shop  0.25
2      Shopping Mall  0.25
3         Restaurant  0.25
4      Women's Store  0.00


----Châteaulin----
                     venue  freq
0  Health & Beauty Service  0.25
1      Arts & Crafts Store  0.25
2             Home Service  0.25
3               Restaurant  0.25
4                   Pal

                  venue  freq
0  Gym / Fitness Center  0.25
1      Football Stadium  0.25
2    Athletics & Sports  0.25
3          Skating Rink  0.25
4         Women's Store  0.00


----Forbach----
               venue  freq
0  French Restaurant   0.2
1      Train Station   0.2
2  Electronics Store   0.2
3                Bar   0.2
4       Soccer Field   0.2


----Forcalquier----
               venue  freq
0  French Restaurant  0.25
1             Museum  0.12
2     Farmers Market  0.12
3     Scenic Lookout  0.12
4              Plaza  0.12


----Fougères----
                        venue  freq
0           French Restaurant  0.22
1                       Hotel  0.22
2                         Bar  0.11
3  Financial or Legal Service  0.11
4                       Plaza  0.11


----Franconville----
            venue  freq
0  Cosmetics Shop   0.5
1    Home Service   0.5
2   Women's Store   0.0
3     Opera House   0.0
4    Outlet Store   0.0


----Fresnes----
              venue  freq
0         

          venue  freq
0        Bakery   0.2
1   Supermarket   0.2
2           Bar   0.2
3         Plaza   0.2
4  Perfume Shop   0.2


----Les Andelys----
         venue  freq
0  Supermarket  0.25
1    Gift Shop  0.12
2    Gastropub  0.12
3   Shoe Store  0.12
4  Men's Store  0.12


----Les Mureaux----
                   venue  freq
0     Photography Studio   1.0
1          Women's Store   0.0
2                 Office   0.0
3  Outdoors & Recreation   0.0
4      Outdoor Sculpture   0.0


----Les Sables-d’Olonne----
               venue  freq
0  French Restaurant  0.13
1              Plaza  0.13
2           Creperie  0.07
3     Ice Cream Shop  0.07
4          Gastropub  0.07


----Lesparre-Médoc----
                     venue  freq
0  Health & Beauty Service  0.25
1      Sporting Goods Shop  0.25
2        Electronics Store  0.25
3              Supermarket  0.25
4                   Office  0.00


----Levallois-Perret----
                 venue  freq
0    French Restaurant  0.12
1         Bu

----Montpellier----
               venue  freq
0  French Restaurant  0.14
1                Bar  0.11
2       Cocktail Bar  0.06
3        Coffee Shop  0.05
4               Café  0.05


----Montreuil----
                venue  freq
0               Hotel  0.16
1               Plaza  0.08
2         Pizza Place  0.08
3       Shopping Mall  0.04
4  Turkish Restaurant  0.04


----Montrouge----
                 venue  freq
0    French Restaurant  0.13
1                Hotel  0.13
2                 Café  0.06
3          Pizza Place  0.06
4  Japanese Restaurant  0.06


----Montélimar----
               venue  freq
0  French Restaurant  0.22
1             Garden  0.11
2                Bar  0.11
3        Supermarket  0.11
4         Restaurant  0.11


----Morlaix----
               venue  freq
0          Bookstore  0.14
1  French Restaurant  0.14
2      Train Station  0.14
3             Bridge  0.14
4              Plaza  0.14


----Mortagne-au-Perche----
                   venue  freq
0            

               venue  freq
0  French Restaurant  0.50
1              Plaza  0.25
2     Ice Cream Shop  0.25
3      Women's Store  0.00
4             Office  0.00


----Provins----
              venue  freq
0  Department Store   0.2
1     Train Station   0.2
2        Restaurant   0.2
3       Pizza Place   0.2
4             Hotel   0.2


----Puteaux----
                  venue  freq
0     French Restaurant  0.21
1                 Hotel  0.17
2   Lebanese Restaurant  0.04
3  Fast Food Restaurant  0.04
4                  Park  0.04


----Périgueux----
                venue  freq
0  Photography Studio  0.25
1       Shopping Mall  0.25
2         Supermarket  0.25
3        Dance Studio  0.25
4       Women's Store  0.00


----Péronne----
            venue  freq
0      Restaurant  0.75
1  History Museum  0.25
2   Women's Store  0.00
3     Opera House  0.00
4    Outlet Store  0.00


----Quimper----
                        venue  freq
0                       Hotel  0.25
1                        C

                   venue  freq
0             Steakhouse   0.2
1                   Park   0.2
2          Train Station   0.2
3   Fast Food Restaurant   0.2
4  Performing Arts Venue   0.2


----Sainte-Geneviève-des-Bois----
           venue  freq
0           Park  0.50
1    Supermarket  0.25
2           Pool  0.25
3  Women's Store  0.00
4   Nudist Beach  0.00


----Sainte-Menehould----
           venue  freq
0          Hotel  0.25
1            Gym  0.25
2  Train Station  0.25
3    Supermarket  0.25
4   Nudist Beach  0.00


----Saintes----
                   venue  freq
0                 Garden  0.12
1            Supermarket  0.12
2                    Bar  0.12
3  Entertainment Service  0.12
4            Pizza Place  0.12


----Salon-de-Provence----
               venue  freq
0  French Restaurant  0.15
1              Plaza  0.15
2                Pub  0.08
3        Beer Garden  0.08
4      Movie Theater  0.08


----Sarcelles----
                     venue  freq
0                     Park  

         venue  freq
0        Hotel  0.25
1       Garden  0.25
2      Brewery  0.25
3  Pizza Place  0.25
4       Office  0.00


----Vigneux-sur-Seine----
                  venue  freq
0      Business Service  0.25
1  Fast Food Restaurant  0.25
2            Taco Place  0.25
3      Asian Restaurant  0.25
4         Women's Store  0.00


----Villefranche-de-Rouergue----
            venue  freq
0           Hotel  0.17
1      Restaurant  0.17
2          Bakery  0.17
3  Clothing Store  0.17
4            Café  0.17


----Villefranche-sur-Saône----
           venue  freq
0          Hotel  0.33
1  Train Station  0.17
2    Supermarket  0.17
3     Restaurant  0.17
4           Café  0.17


----Villejuif----
               venue  freq
0       Concert Hall  0.12
1   Sushi Restaurant  0.12
2  Indian Restaurant  0.12
3      Grocery Store  0.12
4  French Restaurant  0.12


----Villemomble----
               venue  freq
0        Gas Station  0.14
1         Steakhouse  0.14
2   Sushi Restaurant  0.14
3  F

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
#create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = france_grouped['Neighborhood']

for ind in np.arange(france_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(france_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue
0,Abbeville,Hotel
1,Agde,French Restaurant
2,Agen,Multiplex
3,Aix-en-Provence,French Restaurant
4,Aix-les-Bains,Italian Restaurant


In [84]:
#Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

france_grouped_clustering = france_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(france_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 4, 0, 1, 0, 0, 0, 0, 3], dtype=int32)

In [85]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

france_merged = france_cities

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
france_merged = france_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='city')

france_merged.head() # check the last columns!

,city,latitudes,longtitudes,population,Cluster Labels,1st Most Common Venue
0,Paris,48.8566,2.3522,11020000.0,4.0,French Restaurant
1,Nice,43.7034,7.2663,1006402.0,1.0,Hotel
2,Toulouse,43.6045,1.4440,968638.0,0.0,French Restaurant
3,Marseille,43.2964,5.3700,870018.0,0.0,French Restaurant
4,Rennes,48.1147,-1.6794,727357.0,4.0,Creperie


In [86]:
france_merged.loc[france_merged['Cluster Labels'] == 0, france_merged.columns[[0] + list(range(3, france_merged.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue
2,Toulouse,968638.0,0.0,French Restaurant
3,Marseille,870018.0,0.0,French Restaurant
7,Montpellier,607896.0,0.0,French Restaurant
8,Lyon,516092.0,0.0,Bar
9,Rouen,494382.0,0.0,French Restaurant
...,...,...,...,...
452,Mauriac,NaN,0.0,French Restaurant
456,Ambert,NaN,0.0,Optical Shop
457,Forcalquier,NaN,0.0,French Restaurant
468,Florac,NaN,0.0,French Restaurant


In [87]:
france_merged.loc[france_merged['Cluster Labels'] == 1, france_merged.columns[[0] + list(range(3, france_merged.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue
1,Nice,1006402.0,1.0,Hotel
13,Brest,300300.0,1.0,Hotel
37,Belfort,114445.0,1.0,Stadium
40,Montreuil,109897.0,1.0,Hotel
50,Dunkerque,87353.0,1.0,Hotel
...,...,...,...,...
414,Figeac,NaN,1.0,Kids Store
417,Villefranche-de-Rouergue,NaN,1.0,Pub
420,Apt,NaN,1.0,Hotel
422,Avallon,NaN,1.0,Hotel


In [88]:
france_merged.loc[france_merged['Cluster Labels'] == 2, france_merged.columns[[0] + list(range(3, france_merged.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue
224,Les Mureaux,32792.0,2.0,Photography Studio


In [89]:
france_merged.loc[france_merged['Cluster Labels'] == 3, france_merged.columns[[0] + list(range(3, france_merged.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue
26,Villeurbanne,147712.0,3.0,Supermarket
39,Argenteuil,110210.0,3.0,Construction & Landscaping
44,Nanterre,95105.0,3.0,Bakery
51,Aubervilliers,86375.0,3.0,Bakery
79,Pessac,63808.0,3.0,Movie Theater
...,...,...,...,...
445,Mirande,NaN,3.0,Supermarket
453,Clamecy,NaN,3.0,Bakery
455,Nérac,NaN,3.0,Harbor / Marina
462,Yssingeaux,NaN,3.0,Business Service


In [90]:
france_merged.loc[france_merged['Cluster Labels'] == 4, france_merged.columns[[0] + list(range(3, france_merged.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue
0,Paris,11020000.0,4.0,French Restaurant
4,Rennes,727357.0,4.0,Creperie
5,Grenoble,687985.0,4.0,Warehouse Store
6,Nantes,633690.0,4.0,Bar
15,Limoges,283823.0,4.0,Café
...,...,...,...,...
461,Gourdon,NaN,4.0,Tailor Shop
463,Fontainebleau,NaN,4.0,Pizza Place
464,Die,NaN,4.0,Pub
466,Rochechouart,NaN,4.0,Home Service
